In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.utils import np_utils
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Reshape, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from sklearn.metrics import confusion_matrix, classification_report
import itertools
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
from keras.applications.vgg19 import VGG19

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
# Constants
img_rows = 224
img_cols = 224
input_shape = (img_rows,img_cols,3)
epochs = 10
batch_size = 64


In [ ]:
# Get ResNet-50 Model
def getResNet50Model(lastFourTrainable=False):
    resnet_model = ResNet50(weights='imagenet', input_shape=input_shape, include_top=True)
    # Make all layers non-trainable
    for layer in resnet_model.layers[:]:
        layer.trainable = False
    # Add fully connected layer which have 1024 neuron to ResNet-50 model
    output = resnet_model.get_layer('avg_pool').output
    output = Flatten(name='new_flatten')(output)
    output = Dense(units=1024, activation='relu', name='new_fc')(output)
    predictions = Dense(units=50, activation='softmax')(output)
    resnet_model = Model(resnet_model.input, predictions)
    # Make last 4 layers trainable if lastFourTrainable == True
    if lastFourTrainable == True:
        resnet_model.get_layer('conv5_block3_2_bn').trainable = True
        resnet_model.get_layer('conv5_block3_3_conv').trainable = True
        resnet_model.get_layer('conv5_block3_3_bn').trainable = True
        resnet_model.get_layer('new_fc').trainable = True
    # Compile ResNet-50 model
    resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    resnet_model.summary()
    return resnet_model

In [ ]:
traindf = pd.read_csv("../input/landmark-recognition-2021/train.csv")
traindf.head()

In [ ]:
traindf.shape

In [ ]:
landmark_unique = traindf['landmark_id'].unique()
len(landmark_unique)

In [ ]:
landmark_unique[0:50]

In [ ]:
image_ids = []
labels = []
temp_labels = []
i=0
for id_ in landmark_unique[0:50]:
    for iid in traindf['id'][traindf['landmark_id'] == id_]:
        image_ids.append(iid)
        labels.append(id_)
        temp_labels.append(i)
    i = i+1
len(image_ids)

In [ ]:
mainpath = '../input/landmark-recognition-2021/train'
image_path = []
images_pixels = []

for i in range(0,len(image_ids)):
    first_dir = os.path.join(mainpath,image_ids[i][0])
    second_dir = os.path.join(first_dir,image_ids[i][1])
    third_dir = os.path.join(second_dir,image_ids[i][2])
    finalpath = os.path.join(third_dir,image_ids[i]+'.jpg')
    
    img_pix = cv2.imread(finalpath,1)
    images_pixels.append(cv2.resize(img_pix, (224,224)))
    
    image_path.append(finalpath)

In [ ]:
print('Images: ', len(image_path))
print('Image labels: ', len(labels))

In [ ]:
print(temp_labels)

In [ ]:
import random

In [ ]:
fig = plt.gcf()
fig.set_size_inches(16, 16)

next_pix_ = image_path

for i, img_path in enumerate(next_pix_[0:16]):
    
    sp = plt.subplot(5, 4, i + 1)
    sp.axis('Off')

    img = cv2.imread(img_path)
    plt.imshow(img)

plt.show()

In [ ]:
fig = plt.gcf()
fig.set_size_inches(16, 16)

next_pix = image_path
random.shuffle(next_pix)

for i, img_path in enumerate(next_pix[0:12]):
    
    sp = plt.subplot(4, 4, i + 1)
    sp.axis('Off')

    img = cv2.imread(img_path)
    plt.imshow(img)

plt.show()

In [ ]:
shuf = list(zip(images_pixels,temp_labels))
random.shuffle(shuf)

train_data, labels_data = zip(*shuf)
print('Images: ', len(train_data))
print('Image labels: ', len(labels_data))

In [ ]:
print(labels_data)

In [ ]:
'''# Function for plotting the Confusion Matrix
def plotConfusionMatrix(cm, class_names, normalize=True, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.around(cm, decimals=2)
        cm[np.isnan(cm)] = 0.0
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    '''

In [ ]:
"""
# Function for train the given model and get Confusion Matrix
def trainModelAndGetConfusionMatrix(model,train_generator,valid_generator,test_generator,epochs,batch_size):
    # Fit the model
    model.fit_generator(train_generator,
                      epochs=epochs,
                      steps_per_epoch=len(train_generator) // batch_size,
                      validation_data=valid_generator,
                      validation_steps=len(valid_generator) // batch_size)
    # Evaluate the model
    loss_and_metrics = model.evaluate_generator(test_generator, steps=len(test_generator) // batch_size+1)
    print("Test Loss: ", loss_and_metrics[0])
    print("Test Accuracy: ", loss_and_metrics[1])
    test_generator.reset()
    # Get Confusion Matrix and plot it
    Y_pred = model.predict_generator(test_generator, steps=len(test_generator) // batch_size+1)
    y_pred = np.argmax(Y_pred, axis=1)
    class_names = []
    for key in train_generator.class_indices:
        class_names.append(key)
    cr = classification_report(test_generator.classes, y_pred, target_names=class_names)
    print("Classification Report\n" + cr) # Print classification report
    cm = confusion_matrix(test_generator.classes, y_pred)
    plotConfusionMatrix(cm, class_names, normalize=False, title='Confusion Matrix')
    plotConfusionMatrix(cm, class_names, normalize=True, title='Confusion Matrix')
    return model
"""

In [ ]:
train_data = np.array(train_data) #/ 255


In [ ]:
a = np.array(labels_data)
a_pd = pd.get_dummies(a).astype('float32').values 

In [ ]:
print(labels_data[0:6])

In [ ]:
print(a_pd)

In [ ]:
label_data=a_pd

In [ ]:
print(label_data[0:5])

In [ ]:
labels_data.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(train_data, label_data, test_size = 0.3, stratify=label_data,random_state=42)

print("X train data : ", len(X_train))
print("X label data : ", len(X_test))
print("Y test data : ", len(Y_train))
print("Y label data : ", len(Y_test))

In [ ]:
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(train_data, labels_data, test_size = 0.3, stratify=label_data,random_state=42)

print("X train data : ", len(X_train))
print("X label data : ", len(X_test))
print("Y test data : ", len(Y_train))
print("Y label data : ", len(Y_test))

In [ ]:
X_train1.shape

In [ ]:
Y_train1=np.array(Y_train1)
Y_train1.shape

In [ ]:
X_test1.shape

In [ ]:
Y_test1=np.array(Y_test1)
Y_test1.shape

In [ ]:
num_sample=X_train.shape[0]
#20% of the enteries has to be in validation set
validation_freq=int(num_sample*0.2)
#Generating random sample of indices equal to validation_freq
validationlist = random.sample(range(0, num_sample), validation_freq)
traininglist=list(set(range(0,num_sample))-set(validationlist))
print("No interesection between validationlist and traininglist:",set(traininglist).intersection(validationlist))

In [ ]:
train_x_final=[]
train_y_final=[]
for val in traininglist:
    train_x_final.append(X_train[val])
    train_y_final.append(Y_train[val])
train_x_final=np.array(train_x_final)
train_y_final=np.array(train_y_final)
print("Training data shape",train_x_final.shape)
print("Training label shape",train_y_final.shape)

In [ ]:
train_x_final1=[]
train_y_final1=[]
for val in traininglist:
    train_x_final1.append(X_train1[val])
    train_y_final1.append(Y_train1[val])
train_x_final1=np.array(train_x_final1)
train_y_final1=np.array(train_y_final1)
print("Training data shape",train_x_final1.shape)
print("Training label shape",train_y_final1.shape)

In [ ]:
validation_x_final=[]
validation_y_final=[]
for val in validationlist:
    validation_x_final.append(X_train[val])
    validation_y_final.append(Y_train[val])
validation_x_final=np.array(validation_x_final)
validation_y_final=np.array(validation_y_final)
print("Validation data shape",validation_x_final.shape)
print("Validation label shape",validation_y_final.shape)

In [ ]:
validation_x_final1=[]
validation_y_final1=[]
for val in validationlist:
    validation_x_final1.append(X_train1[val])
    validation_y_final1.append(Y_train1[val])
validation_x_final1=np.array(validation_x_final1)
validation_y_final1=np.array(validation_y_final1)
print("Validation data shape",validation_x_final1.shape)
print("Validation label shape",validation_y_final1.shape)

In [ ]:
# plot the training results
def plot_hist(history,title):
    plt.subplot(121)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(str(title)+' accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(122)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(str(title)+' loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

In [ ]:
# Get VGG-16 Model
def getVGG16Model(lastFourTrainable=False):
    vgg_model = VGG16(weights='imagenet', input_shape=input_shape, include_top=True)
    # Make all layers untrainable
    for layer in vgg_model.layers[:]:
        layer.trainable = False
    # Add fully connected layer which have 1024 neuron to VGG-16 model
    output = vgg_model.get_layer('fc2').output
    output = Flatten(name='new_flatten')(output)
    output = Dense(units=1024, activation='relu', name='new_fc')(output)
    output = Dense(units=50, activation='softmax')(output)
    vgg_model = Model(vgg_model.input, output)
    # Make last 4 layers trainable if lastFourTrainable == True
    if lastFourTrainable == True:
        vgg_model.get_layer('block5_conv3').trainable = True
        vgg_model.get_layer('fc1').trainable = True
        vgg_model.get_layer('fc2').trainable = True
        vgg_model.get_layer('new_fc').trainable = True
    # Compile VGG-16 model
    vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    vgg_model.summary()

    return vgg_model

In [ ]:
# Get VGG-19 Model
def getVGG19Model(lastFourTrainable=False):
    vgg_model_19 = VGG19(weights='imagenet', input_shape=input_shape, include_top=True)
    # Make all layers untrainable
    for layer in vgg_model_19.layers[:]:
        layer.trainable = False
    # Add fully connected layer which have 1024 neuron to VGG-16 model
    output = vgg_model_19.get_layer('fc2').output
    output = Flatten(name='new_flatten')(output)
    output = Dense(units=1024, activation='relu', name='new_fc')(output)
    output = Dense(units=50, activation='softmax')(output)
    vgg_model_19 = Model(vgg_model_19.input, output)
    # Make last 4 layers trainable if lastFourTrainable == True
    if lastFourTrainable == True:
        vgg_model_19.get_layer('block5_conv3').trainable = True
        vgg_model_19.get_layer('fc1').trainable = True
        vgg_model_19.get_layer('fc2').trainable = True
        vgg_model_19.get_layer('new_fc').trainable = True
    # Compile VGG-16 model
    vgg_model_19.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    vgg_model_19.summary()

    return vgg_model_19

In [ ]:
vgg_model19_a = getVGG19Model(lastFourTrainable=False)

In [ ]:
vgg_model19_b = getVGG19Model(lastFourTrainable=True)

In [ ]:
# Get ResNet-50 Model with lastFourTrainable=False
resnet_model_a = getResNet50Model(lastFourTrainable=False)

In [ ]:
def score_train(model,test_x,test_y):
    # Score trained model.
    train_scores = model.evaluate(test_x, test_y, verbose=1)
    print('Test loss:', train_scores[0])
    print('Test accuracy:', train_scores[1])

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
print(validation_x_final.shape)
print(validation_y_final.shape)

In [ ]:
import tensorflow

In [ ]:
train_x_final_resnet=tensorflow.keras.applications.resnet.preprocess_input(train_x_final)
validation_x_final_resnet=tensorflow.keras.applications.resnet.preprocess_input(validation_x_final)

In [ ]:
test_x_final_resnet=tensorflow.keras.applications.resnet.preprocess_input(X_test)


RESNET-50-a

In [ ]:
# Train ResNet-50 Model 
#resnet_model_a.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
history1 = resnet_model_a.fit(train_x_final_resnet, train_y_final, epochs=30,validation_data=(validation_x_final_resnet, validation_y_final))
#resnet_model_a = trainModelAndGetConfusionMatrix(resnet_model_a,train_x_final,validation_x_final,X_test,10,64)

In [ ]:
plot_hist(history1,'Resnet-50')

In [ ]:
score_train(resnet_model_a,test_x_final_resnet,Y_test)

Resnet-50-b

In [ ]:
resnet_model_b = getResNet50Model(lastFourTrainable=True)

In [ ]:
# Train ResNet-50 Model 
#resnet_model_a.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
history = resnet_model_b.fit(train_x_final_resnet, train_y_final, epochs=30,validation_data=(validation_x_final_resnet, validation_y_final))
#resnet_model_a = trainModelAndGetConfusionMatrix(resnet_model_a,train_x_final,validation_x_final,X_test,10,64)

In [ ]:
plot_hist(history,'Resnet-50-b')

In [ ]:
score_train(resnet_model_b,test_x_final_resnet,Y_test)

VGG-16-a

In [ ]:
train_x_final_vgg16=tensorflow.keras.applications.vgg16.preprocess_input(train_x_final)
validation_x_final_vgg16=tensorflow.keras.applications.vgg16.preprocess_input(validation_x_final)


In [ ]:
test_x_final_vgg16=tensorflow.keras.applications.vgg16.preprocess_input(X_test)

In [ ]:
vgga=getVGG16Model(lastFourTrainable=False)


In [ ]:
history = vgga.fit(train_x_final_vgg16, train_y_final,validation_data=(validation_x_final_vgg16,validation_y_final), epochs=30)


In [ ]:
plot_hist(history,'VGG-16')

In [ ]:
score_train(vgga,test_x_final_vgg16,Y_test)

In [ ]:
vggb=getVGG16Model(lastFourTrainable=True)

In [ ]:
history_vgg16_b = vggb.fit(train_x_final_vgg16, train_y_final,validation_data=(validation_x_final_vgg16,validation_y_final), epochs=30)

In [ ]:
plot_hist(history_vgg16_b,'Vgg-16-b')

In [ ]:
score_train(vggb,test_x_final_vgg16,Y_test)

VGG-19-a

In [ ]:
train_x_final_vgg19=tensorflow.keras.applications.vgg19.preprocess_input(train_x_final)
validation_x_final_vgg19=tensorflow.keras.applications.vgg19.preprocess_input(validation_x_final)

In [ ]:
test_x_final_vgg19=tensorflow.keras.applications.vgg19.preprocess_input(X_test)


In [ ]:
history_vgg19_a = vgg_model19_a.fit(train_x_final_vgg19, train_y_final,validation_data=(validation_x_final_vgg19,validation_y_final), epochs=30)

In [ ]:
score_train(vgg_model19_a,test_x_final_vgg19,Y_test)

In [ ]:
plot_hist(history_vgg19_a,'vgg-19-a')

VGG-19-b

In [ ]:
history_vgg19_b = vgg_model19_b.fit(train_x_final_vgg19, train_y_final,validation_data=(validation_x_final_vgg19,validation_y_final), epochs=30)

In [ ]:
plot_hist(history_vgg19_b,'VGG-19-b')

In [ ]:
score_train(vgg_model19_b,test_x_final_vgg19,Y_test)

DenseNet

In [ ]:
from keras.applications.densenet import DenseNet121

In [ ]:
# Get DenseNet-121 Model
def getDenseNet121Model(lastFourTrainable=False):
    densenet_model = DenseNet121(weights='imagenet', input_shape=input_shape, include_top=True)
    # Make all layers non-trainable
    for layer in densenet_model.layers[:]:
        layer.trainable = False
    # Add fully connected layer which have 1024 neuron to ResNet-50 model
    output = densenet_model.get_layer('avg_pool').output
    output = Flatten(name='new_flatten')(output)
    output = Dense(units=1024, activation='relu', name='new_fc')(output)
    predictions = Dense(units=50, activation='softmax')(output)
    densenet_model = Model(densenet_model.input, predictions)
    # Make last 4 layers trainable if lastFourTrainable == True
    if lastFourTrainable == True:
        densenet_model.get_layer('conv5_block3_2_bn').trainable = True
        densenet_model.get_layer('conv5_block3_3_conv').trainable = True
        densenet_model.get_layer('conv5_block3_3_bn').trainable = True
        densenet_model.get_layer('new_fc').trainable = True
    # Compile ResNet-50 model
    densenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    densenet_model.summary()
    return densenet_model

In [ ]:
# Get DenseNet-121 Model with lastFourTrainable=False
densenet_model_a = getDenseNet121Model(lastFourTrainable=False)

In [ ]:
train_x_final_densenet=tensorflow.keras.applications.densenet.preprocess_input(train_x_final)
validation_x_final_densenet=tensorflow.keras.applications.densenet.preprocess_input(validation_x_final)

In [ ]:
test_x_final_densenet=tensorflow.keras.applications.densenet.preprocess_input(X_test)

In [ ]:
# Train DENSETNET-121 Model 
#resnet_model_a.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
history_densenet_model_a = densenet_model_a.fit(train_x_final_densenet, train_y_final, epochs=30,validation_data=(validation_x_final_densenet, validation_y_final))
#resnet_model_a = trainModelAndGetConfusionMatrix(resnet_model_a,train_x_final,validation_x_final,X_test,10,64)

In [ ]:
plot_hist(history_densenet_model_a,'Densenet')
score_train(densenet_model_a,test_x_final_densenet,Y_test)

## KNN- Image Retrieval

Using resnet-50-a for feature extraction and KNN

In [ ]:
from keras.models import Model

In [ ]:
feature_model_resnet= Model(inputs=resnet_model_a.input, outputs=resnet_model_a.get_layer('new_fc').output)

In [ ]:
feature_vects=feature_model_resnet.predict(train_x_final_resnet)

In [ ]:
feature_vects.shape

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNeighborsClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
labels_integers=np.argmax(train_y_final, axis=1)

In [ ]:
labels_integers.shape

In [ ]:
knn = KNeighborsClassifier()
knn.fit(feature_vects,train_y_final1)

In [ ]:
test_vects=feature_model_resnet.predict(test_x_final_resnet)

In [ ]:
test_vects.shape

In [ ]:
Y_test.shape

In [ ]:
test_label_integers=np.argmax(Y_test, axis=1)

In [ ]:
test_label_integers.shape

In [ ]:
print(knn.score(test_vects,Y_test1))
#ypred=knn.predict(fin_test_img)

RETREIVAL of FIRST 7 IMAGES

In [ ]:
knn7 = KNeighborsClassifier(n_neighbors = 7)
knn7.fit(feature_vects,train_y_final1)

In [ ]:
indices=knn7.kneighbors(test_vects, return_distance=False)

In [ ]:
indices.shape

In [ ]:
np.max(indices)

In [ ]:
np.min(indices)

In [ ]:
def show_neighbors(orig, neighbors):
    f, axarr = plt.subplots(4, 2)
    for i, ax in enumerate(axarr.flatten()):
        if i == 0:
            ax.set_title("Query image")
            ax.imshow(orig)
        else:
            ax.set_title(f"Neighbor {i}")
            ax.imshow(train_x_final[neighbors[i-1]])
        ax.set_yticklabels([])
        ax.set_xticklabels([])
    plt.tight_layout()
    plt.show()

In [ ]:
indices=knn7.kneighbors(test_vects, return_distance=False)
for i in range(0,5):
    show_neighbors(X_test[i],indices[i])
    
    

In [ ]:
ypred=knn7.predict(test_vects)


In [ ]:
from sklearn import metrics 
print(metrics.classification_report(Y_test1,ypred))

In [ ]:
prec1={}
recal1={}
for i in range(0,50):
    prec1[i]=[]
    recal1[i]=[]
    

In [ ]:
for k in range(1,10):
    knn=KNeighborsClassifier(n_neighbors = k)
    knn.fit(feature_vects,train_y_final1)
    ypred=knn.predict(test_vects)
    report=metrics.classification_report(Y_test1,ypred,output_dict=True)
    dt=pd.DataFrame(report).transpose()
    #print(dt)
    for i in range(50):
        #print(i,dt["precision"][i])
        prec1[i].append(dt["precision"][i])
        recal1[i].append(dt["recall"][i])

In [ ]:
print(prec1)

In [ ]:
print(recal1)

In [ ]:
for i in range(0,50):
    prec1[i].append(1.0)
    recal1[i].append(0.0)

In [ ]:
print(prec1)

In [ ]:
print(recal1)

In [ ]:
ap={}
for i in range(0,50):
    precisions=prec1[i]
    recalls=recal1[i]
    precisions = np.array(precisions)
    recalls = np.array(recalls)
    ap[i]=np.sum((recalls[:-1] - recalls[1:]) * precisions[:-1])

In [ ]:
print(ap)

In [ ]:
sum=0
for i in range(0,50):
    sum+=ap[i]

In [ ]:
print(sum)

In [ ]:
print(sum/50)